# Day 1

## Part 1

In [6]:
with open('inputs/day1.txt') as f:
    s = f.read()[:-1].split('\n')
    
sum([int(n) for n in s])

525

## Part 2

In [10]:
seen = set()

frequencies = [int(n) for n in s]

i = 0
f = 0
while f not in seen:
    seen.add(f)
    f += frequencies[i%len(frequencies)]
    i += 1
    
f

75749

# Day 2

## Part 1

In [21]:
with open('inputs/day2.txt') as f:
    s = f.read()[:-1].split('\n')

twos = 0
threes = 0
for box in s:
    two = False
    three = False
    for c in range(ord('a'), ord('z')+1):
        count = box.count(chr(c))
        if count == 2:
            two = True
        if count == 3:
            three = True
    if two:
        twos += 1
    if three:
        threes += 1
            
twos*threes

7350

## Part 2

In [25]:
def hamming(s1,s2):
    d = 0
    for i in range(len(s1)):
        if s1[i] != s2[i]:
            d += 1
    return d

for i in range(len(s)):
    for j in range(i+1,len(s)):
        if hamming(s[i],s[j]) == 1:
            s1 = s[i]
            s2 = s[j]
            
''.join([s1[i] for i in range(len(s1)) if s1[i] == s2[i]])

'wmlnjevbfodamyiqpucrhsukg'

# Day 3

## Part 1

In [45]:
with open('inputs/day3.txt') as f:
    s = f.read()[:-1].split('\n')
    
def parse(claim):
    pos = claim.split('@ ')[1].split(':')[0]
    pos = (int(pos.split(',')[0]), int(pos.split(',')[1]))
    size = claim.split(': ')[-1]
    size = (int(size.split('x')[0]), int(size.split('x')[1]))
    return (pos, (pos[0]+size[0], pos[1]+size[1]))
    
claims = {parse(c): c.split(' ')[0][1:] for c in s}
claimed = set()
double_claimed = set()

for c in claims:
    for x in range(c[0][0], c[1][0]):
        for y in range(c[0][1], c[1][1]):
            if (x,y) in claimed:
                double_claimed.add((x,y))
            else:
                claimed.add((x,y))
            
len(double_claimed)

111935

## Part 2

In [46]:
for c in claims:
    match = True
    for x in range(c[0][0], c[1][0]):
        for y in range(c[0][1], c[1][1]):
            if (x,y) in double_claimed:
                match = False
    if match:
        break
        
claims[c]

'650'

# Day 4

## Part 1

In [70]:
with open('inputs/day4.txt') as f:
    s = f.read()[:-1].split('\n')
    
s.sort()
current_guard = None
last_sleep = None
last_wake = None
sleeping = False
shifts = {}

def parse(entry):
    global current_guard
    global last_sleep
    global last_wake
    global sleeping
    
    minute = int(entry.split(':')[1].split(']')[0])
    hour = int(entry.split(' ')[1].split(':')[0])
    if hour != 0:
        minute = 0
    if 'Guard' in entry:
        if sleeping:
            for t in range(last_sleep, 60):
                shifts[current_guard][len(shifts[current_guard])-1].append(t)
        sleeping = False
        
        current_guard = int(entry.split('#')[1].split(' ')[0])
        if current_guard not in shifts:
            shifts[current_guard] = {}
        shifts[current_guard][len(shifts[current_guard])] = []
    else:
        if 'asleep' in entry:
            last_sleep = minute
            sleeping = True
        else:
            sleeping = False
            last_wake = minute
            for t in range(last_sleep, last_wake):
                shifts[current_guard][len(shifts[current_guard])-1].append(t)
        
for entry in s:
    parse(entry)
    
sleeps = {guard: sum([len(shifts[guard][s]) for s in shifts[guard]]) for guard in shifts}
most_sleeping = max(sleeps, key=sleeps.get)

sleep_minutes = []
for shift in shifts[most_sleeping]:
    sleep_minutes += shifts[most_sleeping][shift]

minute_sleep_count = [sleep_minutes.count(t) for t in range(60)]
max_slept_minute = max(minute_sleep_count)
most_slept_minute = minute_sleep_count.index(max_slept_minute)

most_sleeping*most_slept_minute

8421

## Part 2

In [75]:
frequencies = {}

for guard in shifts:
    sleep_minutes = []
    for shift in shifts[guard]:
        sleep_minutes += shifts[guard][shift]
    frequencies[guard] = [sleep_minutes.count(t) for t in range(60)]
    
max_guard_minute = 0
guard = 0
minute = 0
for g in frequencies:
    for m in range(60):
        if frequencies[g][m] > max_guard_minute:
            max_guard_minute = frequencies[g][m]
            guard = g
            minute = m
            
guard*minute

83359

# Day 5

## Part 1

In [108]:
with open('inputs/day5.txt') as f:
    s = f.read()[:-1]

units = [chr(c) for c in range(ord('a'), ord('z')+1)]

def react(s):
    start = ''
    result = s
    while len(start) != len(result):
        start = result
        for u in units:
            while u + u.upper() in result:
                result = result.replace(u + u.upper(), '')
            while u.upper() + u in result:
                result = result.replace(u.upper() + u, '')
    return result
    
len(react(s))

11298

## Part 2

In [109]:
min_length = len(s)
for u in units:
    poly = ''.join([c for c in s if c != u and c != u.upper()])
    result = react(poly)
    if len(result) < min_length:
        min_length = len(result)
        
min_length

5148

# Day 6

## Part 1

In [24]:
import queue

with open('inputs/day6.txt') as f:
    s = f.read()[:-1].split('\n')
    
coords = {i: (int(s[i].split(', ')[0]), int(s[i].split(', ')[1])) for i in range(len(s))}
closest = {}

limit = 10000

def distances(coord):    
    return {c: abs(coords[c][0]-coord[0]) + abs(coords[c][1]-coord[1]) for c in coords}

def closest(coord):    
    d = distances(coord)
    min_d = min(d.values())
    d = {c: d[c] for c in d if d[c] == min_d}
    return next(iter(d)) if len(d) == 1 else -1
    
def bfs(source, coord, visited, matching, q):
    if coord in visited:
        return
    
    visited.add(coord)
    
    if closest(coord) == source:
        matching.add(coord)

        (x,y) = coord
        if (x,y-1) not in visited:
            q.put((x,y-1))
        if (x,y+1) not in visited:
            q.put((x,y+1))
        if (x-1,y) not in visited:
            q.put((x-1,y))
        if (x+1,y) not in visited:
            q.put((x+1,y))
            
def discover(source):
    q = queue.Queue()
    matching = set()
    visited = set()
    q.put(coords[source])
    while not q.empty() and len(matching) < limit:
        bfs(source, q.get(), visited, matching, q)
    if not q.empty():
        return -1
    else:
        return len(matching)
    
areas = {c: discover(c) for c in coords}

max(areas.values())

4060

## Part 2

In [32]:
matches = 0
for x in range(500):
    for y in range(500):
        if sum(distances((x,y)).values()) < 10000:
            matches += 1
            
matches

36136

# Day 7

## Part 1

In [48]:
with open('inputs/day7.txt') as f:
    s = f.read()[:-1].split('\n')

deps = [(d.split(' ')[1], d.split(' ')[-3]) for d in s]
    
dependencies = {}
for dep in deps:
    if dep[0] not in dependencies:
        dependencies[dep[0]] = []
    if dep[1] not in dependencies:
        dependencies[dep[1]] = []
    dependencies[dep[1]].append(dep[0])
    
order = []

while len(dependencies) != len(order):
    executable = [d for d in dependencies if len(dependencies[d]) == 0 and d not in order]
    executable.sort()
    e = executable[0]
    order.append(e)
    dependencies = {d: [d1 for d1 in dependencies[d] if d1!= e] for d in dependencies}
    
''.join(order)

'FMOXCDGJRAUIHKNYZTESWLPBQV'

## Part 2

In [60]:
finish_times = {}
startable_time = {}
free_workers = 5

dependencies = {}
for dep in deps:
    if dep[0] not in dependencies:
        dependencies[dep[0]] = []
    if dep[1] not in dependencies:
        dependencies[dep[1]] = []
    dependencies[dep[1]].append(dep[0])

def task_time(task_id):
    return ord(task_id) - 4

def free_worker():
    global free_workers
    free_workers = min(free_workers+1, 5)
    
def tick(t):
    global dependencies
    finished = [f for f in finish_times if finish_times[f] == t]
    for f in finished:
        free_worker()
        dependencies = {d: [d1 for d1 in dependencies[d] if d1!= f] for d in dependencies}
    executable = [d for d in dependencies if len(dependencies[d]) == 0 and d not in finish_times]
    executable.sort()
    for i in range(min(free_workers, len(executable))):
        e = executable[0]
        executable = executable[1:]
        finish_times[e] = t + task_time(e)
        
        
t = 0
while len(dependencies) != len(finish_times):
    tick(t)
    t += 1
    
max(finish_times.values())

1053

# Day 8

## Part 1

In [60]:
with open('inputs/day8.txt') as f:
    s = f.read()[:-1]
    
tree = [int(n) for n in s.split(' ')]

class Node:
    def __init__(self, parent=None):
        self.parent = parent
        self.children = []
        self.metadata = []
        
    def meta_sum(self):
        a = sum(self.metadata)
        for child in self.children:
            a += child.meta_sum()
        return a
    
    def value(self):
        if len(self.children) == 0:
            return sum(self.metadata)
        else:
            a = 0
            for m in self.metadata:
                if m <= len(self.children):
                    a += self.children[m-1].value()
            return a
                
        
root = Node()
current = root
offset = 0
while len(tree) > 0:
    while tree[offset] != 0:
        offset += 2
        current.children.append(Node(current))
        current = current.children[-1]
        
    for i in range(offset+2, offset+2+tree[offset+1]):
        current.metadata.append(tree[i])
        
    tree = tree[:offset] + tree[offset+2+tree[offset+1]:]
    
    if offset != 0:
        offset -= 2
        tree[offset] -= 1
        current = current.parent
        
root.meta_sum()

46096

## Part 2

In [61]:
root.value()

24820

# Day 9

## Part 1

In [69]:
elves = 416
max_marble = 71617

class Node:
    def __init__(self, value, prev_node = None, next_node = None):
        self.value = value
        if prev_node is not None:
            self.prev = prev_node
            prev_node.next = self
        else:
            self.prev = self
        if next_node is not None:
            self.next = next_node
            next_node.prev = self
        else:
            self.next = self
        
    def pick(self):
        self.prev.next = self.next
        self.next.prev = self.prev
        return self.value

players = {i: 0 for i in range(elves)}
current = Node(0)

for i in range(0, max_marble):
    if (i+1) % 23 != 0:
        current = current.next
        current = Node(i+1, current, current.next)
    else:
        players[i%elves] += i+1
        for j in range(7):
            current = current.prev
        players[i%elves] += current.pick()
        current = current.next
        
max(players.values())

436720

## Part 2

In [72]:
max_marble = 7161700

players = {i: 0 for i in range(elves)}
current = Node(0)

for i in range(0, max_marble):
    if (i+1) % 23 != 0:
        current = current.next
        current = Node(i+1, current, current.next)
    else:
        players[i%elves] += i+1
        for j in range(7):
            current = current.prev
        players[i%elves] += current.pick()
        current = current.next
        
max(players.values())

3527845091

# Day 10

## Part 1

In [121]:
with open('inputs/day10.txt') as f:
    s = f.read()[:-1].split('\n')
    
def parse(string):
    numbers = string[:-1].replace('position=<', '').replace('> velocity=<', ', ').split(', ')
    return ((int(numbers[0]), int(numbers[1])),(int(numbers[2]), int(numbers[3])))

stars = [(parse(star)[0], parse(star)[1])  for star in s]

def move(stars):
    return [((star[0][0] + star[1][0], star[0][1] + star[1][1]), star[1]) for star in stars]

def draw(stars):
    stars = {s[0] for s in stars}
    min_x = min_y = max_x = max_y = None
    for star in stars:
        if min_x is None or min_x > star[0]:
            min_x = star[0]
        if max_x is None or max_x < star[0]:
            max_x = star[0]
        if min_y is None or min_y > star[1]:
            min_y = star[1]
        if max_y is None or max_y < star[1]:
            max_y = star[1]
    
    return [''.join(['#' if (x,y) in stars else '.' for x in range(min_x, max_x+1)]) for y in range(min_y, max_y+1)]

pattern_size = 6
def check_for_message(stars):
    stars = {s[0] for s in stars}
    for star in stars:
        (x,y) = star
        
        match_x = True
        match_y = True
        
        for i in range(1,pattern_size+1):
            if match_x:
                match_x = (x+i, y) in stars
            if match_y:
                match_y = (x, y+i) in stars
            if not (match_x or match_y):
                break
        
        if match_x or match_y:
            return True
    return False
    
while not check_for_message(stars):
    stars = move(stars)

draw(stars)

['#####...#####...#....#..#....#..#....#..######..######..#####.',
 '#....#..#....#..##...#..##...#..#....#..#............#..#....#',
 '#....#..#....#..##...#..##...#...#..#...#............#..#....#',
 '#....#..#....#..#.#..#..#.#..#...#..#...#...........#...#....#',
 '#####...#####...#.#..#..#.#..#....##....#####......#....#####.',
 '#..#....#.......#..#.#..#..#.#....##....#.........#.....#..#..',
 '#...#...#.......#..#.#..#..#.#...#..#...#........#......#...#.',
 '#...#...#.......#...##..#...##...#..#...#.......#.......#...#.',
 '#....#..#.......#...##..#...##..#....#..#.......#.......#....#',
 '#....#..#.......#....#..#....#..#....#..#.......######..#....#']

## Part 2

In [122]:
stars = [(parse(star)[0], parse(star)[1])  for star in s]

moves = 0
while not check_for_message(stars):
    moves += 1
    stars = move(stars)
    
moves

10946

# Day 11

## Part 1

In [145]:
sn = 3031

def power(x,y):
    rack_id = x+10
    power_level = rack_id * y
    power_level += sn
    power_level *= rack_id
    power_level //= 100
    power_level %= 10
    power_level -= 5
    return power_level
    
cells = {(x,y): power(x,y) for y in range(1,301) for x in range(1,301)}
cache = {}

def area_power(x,y,size):
    if (x,y,size) in cache:
        return cache[(x,y,size)]
    if size == 1:
        return cells[(x,y)]
    power = area_power(x,y,size-1)
    for i in range(size):
        power += cells[(x+size-1,y+i)]
        power += cells[(x+i,y+size-1)]
    power -= cells[(x+size-1,y+size-1)]
            
    cache[(x,y,size)] = power
    return power

def max_coords(size):
    max_power = -1000000
    max_x = max_y = 0
    for x in range(1,301-(size-1)):
        for y in range(1, 301-(size-1)):
            power = area_power(x,y,size)
            if power > max_power:
                max_power = power
                max_x = x
                max_y = y
    return max_power, max_x, max_y


max_coords(3)[1:]

(21, 76)

## Part 2

In [159]:
max_power = -1000000
max_x = max_y = max_size = 0
for size in range(1, 301):
    power,x,y = max_coords(size)
    if power > max_power:
        max_power = power
        max_x = x
        max_y = y
        max_size = size
    
(max_x, max_y, max_size)

(234, 108, 16)

# Day 12

## Part 1

In [192]:
with open('inputs/day12.txt') as f:
    s = f.read()[:-1].split('\n')
    
init = s[0].replace('initial state: ', '')
rules = {r.split(' => ')[0]: r.split(' => ')[1] for r in s[2:]}

def surroundings(state,i):
    l = len(state)
    if i == 0:
        return '..' + state[:3]
    if i == 1:
        return '.' + state[:4]
    if i == l-1:
        return state[l-3:] + '..'
    if i == l-2:
        return state[l-4:] + '.'
    return state[i-2:i+3]

def next_state(state, start):
    new_state = '..' + state + '..'
    start -= 2
    new_state = ''.join([rules[surroundings(new_state, i)] for i in range(len(new_state))])
    first_plant = new_state.index('#')
    start += first_plant
    return new_state[first_plant:new_state.rindex('#')+1], start

state = init
start = 0
generations = 20
for i in range(generations):
    state, start = next_state(state, start)
    
def plants(state, start):
    a = 0
    for i in range(len(state)):
        if state[i] == '#':
            a += start+i
    return a

plants(state, start)

3258

## Part 2

In [213]:
generations = 50000000000
stability_limit = 100
last_diff = 0
last_sum = 0
stability = 0

state = init
start = 0
for i in range(generations):
    state, start = next_state(state, start)
    a = plants(state, start)
    diff = a - last_sum
    if diff == last_diff:
        stability += 1
    last_diff = diff
    last_sum = a
    if stability >= stability_limit:
        break
    
last_sum + last_diff*(generations-i-1)

3600000002022

# Day 13

## Part 1

In [345]:
with open('inputs/day13.txt') as f:
    s = f.read()[:-1]
    
tracks = s.split('\n')
left = {(0,-1): (-1,0), (-1,0): (0,1), (0,1): (1,0), (1,0): (0,-1)}
right = {left[d]: d for d in left}
directions = {'<': (-1,0), '>': (1,0), '^': (0,-1), 'v': (0,1)}
turning_order = [lambda d: left[d], lambda d: d, lambda d: right[d]]
    
def cart_at(x,y,carts):
    return [c for c in carts if carts[c].x == x and carts[c].y == y]
    
class Cart:
    def __init__(self,cart_id,x,y,direction):
        self.x = x
        self.y = y
        self.direction = direction
        self.state = 0
        self.cart_id = cart_id
        
    def __repr__(self):
        return '({}, {})'.format(self.x, self.y)
        
    def crashed(self,carts):
        carts_here = cart_at(self.x, self.y,carts)
        return carts_here if len(carts_here) > 1 else []
        
    def move(self):
        self.x += self.direction[0]
        self.y += self.direction[1]
        if tracks[self.y][self.x] == '\\' and self.direction[0] == 0:
            self.direction = left[self.direction]
        elif tracks[self.y][self.x] == '\\' and self.direction[1] == 0:
            self.direction = right[self.direction]
        elif tracks[self.y][self.x] == '/' and self.direction[0] == 0:
            self.direction = right[self.direction]
        elif tracks[self.y][self.x] == '/' and self.direction[1] == 0:
            self.direction = left[self.direction]
        elif tracks[self.y][self.x] == '+':
            self.direction = turning_order[self.state](self.direction)
            self.state = (self.state + 1) % 3
        
        
carts = {}
for y in range(len(tracks)):
    for x in range(len(tracks[0])):
        if tracks[y][x] in '<>^v':
            carts[len(carts)] = Cart(len(carts),x,y,directions[tracks[y][x]])
    tracks[y] = tracks[y].replace('^', '|').replace('v', '|').replace('<','-').replace('>','-')
    
safe = True
def move_carts():
    global safe
    cart_order = [(cart, carts[cart].x, carts[cart].y) for cart in carts]
    cart_order.sort(key=lambda cart: cart[2])
    cart_order.sort(key=lambda cart: cart[1])
    
    crashed = []
    carts_new = {c:carts[c] for c in carts}
    for cart in [c[0] for c in cart_order]:
        if cart not in crashed:
            carts_new[cart].move()
            crash = carts_new[cart].crashed(carts_new)
            if len(crash) != 0 and safe:
                print(carts_new[cart].x, carts_new[cart].y)
                safe = False
            crashed += crash
            carts_new = {c: carts_new[c] for c in carts_new if c not in crashed}
    return carts_new

while safe:
    carts = move_carts()

123 18


## Part 2

In [346]:
tracks = s.split('\n')
carts = {}
for y in range(len(tracks)):
    for x in range(len(tracks[0])):
        if tracks[y][x] in '<>^v':
            carts[len(carts)] = Cart(len(carts),x,y,directions[tracks[y][x]])
    tracks[y] = tracks[y].replace('^', '|').replace('v', '|').replace('<','-').replace('>','-')

while len(carts) != 1:
    carts = move_carts()

carts

{2: (71, 123)}

# Day 14

## Part 1

In [360]:
recipe_count = 440231
recipes = ['3','7']
e0 = 0
e1 = 1

def create(recipes, e0, e1):
    new_recipe = int(recipes[e0])+int(recipes[e1])
    if new_recipe < 10:
        recipes.append(str(new_recipe))
    else:
        recipes.append(str(new_recipe//10))
        recipes.append(str(new_recipe%10))
    e0 = (e0 + int(recipes[e0]) + 1) % len(recipes)
    e1 = (e1 + int(recipes[e1]) + 1) % len(recipes)
    return recipes, e0, e1

while len(recipes) < recipe_count + 10:
    recipes, e0, e1 = create(recipes, e0, e1)
    
''.join(recipes[recipe_count:recipe_count+10])

'1052903161'

## Part 2

In [378]:
recipe_count_str = str(recipe_count)
recipes = ['3','7']
e0 = 0
e1 = 1

while recipe_count_str not in ''.join(recipes[-10:]):
    recipes, e0, e1 = create(recipes, e0, e1)
    
''.join(recipes).index(recipe_count_str)

20165504

# Day 15

## Part 1

In [121]:
import queue

with open('inputs/day15.txt') as f:
    s = f.read()[:-1].split('\n')

characters = {'E': {}, 'G': {}}
ground = set()
    
def adjecent(position):
    adj = [(position[0],   position[1]-1), \
           (position[0]-1, position[1]), \
           (position[0]+1, position[1]), \
           (position[0],   position[1]+1)]
    return [p for p in adj if p in ground]
    
def adjecent_list(positions):
    adj = set()
    for p in positions:
        adj.update(adjecent(p))
    return adj

visited = {}
q = queue.Queue()
def bfs(path):
    (position, first, length) = path
    if position not in visited: 
        visited[position] = (first, length)
        adj = adjecent(position)
        for pos in adj:
            if pos not in visited and pos not in characters['E'] and pos not in characters['G']:
                f = pos if first is None else first
                q.put((pos, f, length+1))
    
def next_on_shortest_path(position, targets):
    global q
    global visited
    
    visited = {}
    q = queue.Queue()
    q.put((position, None, 0))
    while not q.empty():
        bfs(q.get())
    
    target_distances = {v: visited[v] for v in visited if v in targets and v != position}
    if len(target_distances) > 0:
        min_dist = target_distances[min(target_distances, key=lambda x: target_distances[x][1])][1]
        min_distance_targets = [t for t in target_distances if target_distances[t][1] == min_dist]
        min_distance_targets.sort(key=lambda t: t[0])
        min_distance_targets.sort(key=lambda t: t[1])
        target = min_distance_targets[0]
        
        return target_distances[target][0]
    else:
        return position

class Character:
    def __init__(self, position):
        self.position = position
        self.hp = 200
        
    def __repr__(self):
        return '<Type: {}, HP: {}, Position: {}>'.format(self.character_type, self.hp, self.position)
    
    def move(self, targets):
        new_position = next_on_shortest_path(self.position, targets)
        if new_position != self.position:
            characters[self.character_type][new_position] = self
            del characters[self.character_type][self.position]
            self.position = new_position        
        
    def pick_target(self):
        targets = [t for t in adjecent(self.position) if t in characters[self.enemy_type]]
        hps = {enemy: characters[self.enemy_type][enemy].hp for enemy in targets}
        min_hp = hps[min(hps, key=hps.get)]
        min_hps = set(enemy for enemy in hps if hps[enemy] == min_hp)
        reading_order = adjecent(self.position)
        for p in reading_order:
            if p in min_hps:
                return characters[self.enemy_type][p]
    
    def attack(self, character):
        character.damage(self.ap)
    
    def damage(self, damage):
        self.hp -= damage
        if self.hp <= 0:
            self.die()
            
    def die(self):
        del characters[self.character_type][self.position]
    
    def act(self):
        if self.hp > 0:
            attack_targets = characters[self.enemy_type]
            move_targets = adjecent_list(attack_targets)
            if self.position not in move_targets:
                self.move(move_targets)
            if self.position in move_targets:
                target = self.pick_target()
                self.attack(target)

class Elf(Character):
    def __init__(self, position, ap):
        self.character_type = 'E'
        self.enemy_type = 'G'
        self.ap = ap
        super().__init__(position)
        
class Goblin(Character):
    def __init__(self, position):
        self.character_type = 'G'
        self.enemy_type = 'E'
        self.ap = 3
        super().__init__(position)

def play(elf_ap):
    global characters
    
    characters = {'E': {}, 'G': {}}
    for y in range(len(s)):
        for x in range(len(s[0])):
            if s[y][x] != '#':
                ground.add((x,y))
            if s[y][x] == 'E':
                characters['E'][(x,y)] = Elf((x,y),elf_ap)
            if s[y][x] == 'G':
                characters['G'][(x,y)] = Goblin((x,y))

    elves = len(characters['E'])
    rounds = 0
    while len(characters['E']) != 0 and len(characters['G']) != 0:
        all_characters = []
        all_characters += [(c[0], c[1], characters['E'][c]) for c in characters['E']]
        all_characters += [(c[0], c[1], characters['G'][c]) for c in characters['G']]
        all_characters.sort(key=lambda c: c[0])
        all_characters.sort(key=lambda c: c[1])
        for character in all_characters:
            character[2].act()
        if len(characters['E']) != 0 and len(characters['G']) != 0:
            rounds += 1
            
    hps = [char.hp for char in list(characters['E'].values()) + list(characters['G'].values())]
            
    return (len(characters['E']) == elves, rounds*sum(hps))
        
play(3)[1]

214731

## Part 2

In [124]:
elf_ap = 3
outcome = play(elf_ap)
while not outcome[0]:
    elf_ap += 1
    outcome = play(elf_ap)
    
outcome[1]

53222

# Day 16

## Part 1

In [177]:
with open('inputs/day16.txt') as f:
    s = f.read()[:-1]
    
sample = [t.split('\n') for t in s.split('\n\n\n')[0].split('\n\n')]
    
def addr(a,b,c,registers):
    registers[c] = registers[a] + registers[b]
    return registers
    
def addi(a,b,c,registers):
    registers[c] = registers[a] + b
    return registers
    
def mulr(a,b,c,registers):
    registers[c] = registers[a] * registers[b]
    return registers
    
def muli(a,b,c,registers):
    registers[c] = registers[a] * b
    return registers
    
def banr(a,b,c,registers):
    registers[c] = registers[a] & registers[b]
    return registers
    
def bani(a,b,c,registers):
    registers[c] = registers[a] & b
    return registers
    
def borr(a,b,c,registers):
    registers[c] = registers[a] | registers[b]
    return registers
    
def bori(a,b,c,registers):
    registers[c] = registers[a] | b
    return registers
    
def setr(a,b,c,registers):
    registers[c] = registers[a]
    return registers
    
def seti(a,b,c,registers):
    registers[c] = a
    return registers
    
def gtir(a,b,c,registers):
    registers[c] = int(a > registers[b])
    return registers
    
def gtri(a,b,c,registers):
    registers[c] = int(registers[a] > b)
    return registers
    
def gtrr(a,b,c,registers):
    registers[c] = int(registers[a] > registers[b])
    return registers
    
def eqir(a,b,c,registers):
    registers[c] = int(a == registers[b])
    return registers
    
def eqri(a,b,c,registers):
    registers[c] = int(registers[a] == b)
    return registers
    
def eqrr(a,b,c,registers):
    registers[c] = int(registers[a] == registers[b])
    return registers

ops = {addr, addi, mulr, muli, banr, bani, borr, bori, setr, seti, gtir, gtri, gtrr, eqir, eqri, eqrr}
def test(args):
    [opcode, a, b, c] = [int(v) for v in args[1].split(' ')]
    after = [int(v) for v in args[2][9:-1].split(', ')]
    
    correct = 0
    for op in ops:
        before = [int(v) for v in args[0][9:-1].split(', ')]
        if op(a,b,c,before) == after:
            correct += 1
            
    return correct
    
gte3 = 0
for t in sample:
    if test(t) >= 3:
        gte3 += 1
    
gte3

500

## Part 2

In [186]:
op_codes = {code: {op for op in ops} for code in range(16)}

def test(args):
    [op_code, a, b, c] = [int(v) for v in args[1].split(' ')]
    after = [int(v) for v in args[2][9:-1].split(', ')]
    
    incorrect = set()
    for op in op_codes[op_code]:
        before = [int(v) for v in args[0][9:-1].split(', ')]
        if op(a,b,c,before) != after:
            incorrect.add(op)
            
    op_codes[op_code] = {op for op in op_codes[op_code] if op not in incorrect}
    if len(op_codes[op_code]) == 1:
        taken = next(iter(op_codes[op_code]))
        for code in op_codes:
            if code != op_code:
                op_codes[code] = {op for op in op_codes[code] if op != taken}

for t in sample:
    test(t)
    
op_codes = {op: next(iter(op_codes[op])) for op in op_codes}
    
test_program = s.split('\n\n\n\n')[1].split('\n')

def execute(program):
    registers = [0,0,0,0]
    for operation in program:
        [op_code, a, b, c] = [int(v) for v in operation.split(' ')]
        op_codes[op_code](a,b,c,registers)
    return registers
    

execute(test_program)[0]

533

# Day 17

## Part 1

In [123]:
import queue

with open('inputs/day17.txt') as f:
    s = f.read()[:-1].split('\n')
    
def build_level(clay):
    for line in s:
        for coord in line.split(', '):
            if '..' not in coord:
                n = int(coord[2:])
                if 'x' in coord:
                    x = range(n,n+1)
                else:
                    y = range(n,n+1)
            else:
                r = coord[2:].split('..')
                low = int(r[0])
                high = int(r[1])
                if 'x' in coord:
                    x = range(low,high+1)
                else:
                    y = range(low,high+1)
        for x_i in x:
            for y_i in y:
                clay.add((x_i,y_i))
        
def flow(source,clay,water,running_water,limit):
    x,y = source
    while (x,y+1) not in clay and (x,y+1) not in water and y+1 <= limit:
        y += 1
        if (x,y+1) not in running_water:
            running_water_q.put((x,y))
        running_water.add((x,y))
    return x,y
    
def fill(source,clay,water,running_water,running_water_q,limit):
    bottom = flow(source,clay,water,running_water,limit)
    if bottom == source:
        x,y = source
        if (x,y+1) in water or (x,y+1) in clay:
            path = set()
            path.add((x,y))
            running = False
            while (x-1,y) not in clay and ((x-1,y+1) in clay or (x-1,y+1) in water):
                x -= 1
                path.add((x,y))
            if not ((x-1,y+1) in clay or (x-1,y+1) in water):
                running = True
            if (x-1,y) not in clay:
                path.add((x-1,y))
                x_min = x-1
            x,y = source
            while (x+1,y) not in clay and ((x+1,y+1) in clay or (x+1,y+1) in water):
                x += 1
                path.add((x,y))
            if not ((x+1,y+1) in clay or (x+1,y+1) in water):
                running = True
            if (x+1,y) not in clay:
                path.add((x+1,y))
                x_max = x+1
            if running:
                for w in path:
                    if w not in running_water:
                        running_water_q.put(w)
                running_water.update(path)
            else:
                for w in path:
                    if w in running_water:
                        running_water.remove(w)
                water.update(path)
    
           
clay = set()
spring = (500,0)
water = set()
running_water = set()
running_water_q = queue.LifoQueue()
build_level(clay)
min_y = min([c[1] for c in clay])
max_y = max([c[1] for c in clay])
running_water_q.put(spring)

while not running_water_q.empty():
    fill(running_water_q.get(),clay,water,running_water,running_water_q,max_y)

len([w for w in water if w[1] in range(min_y, max_y+1)]) + len([w for w in running_water if w[1] in range(min_y, max_y+1)])

31383

## Day 2

In [125]:
len(water)

25376

# Day 18

## Part 1

In [140]:
with open('inputs/day18.txt') as f:
    s = f.read()[:-1].split('\n')
    
lumber = [list(row) for row in s]

def adjecent(x,y,lumber):
    types = {'.': 0, '|': 0, '#': 0}
    for i in range(-1,2):
        for j in range(-1,2):
            if x+i in range(50) and y+j in range(50) and not (i == 0 and j == 0):
                types[lumber[y+j][x+i]] += 1
                
    return types

def next_state(x,y,lumber):
    adj = adjecent(x,y,lumber)
    current = lumber[y][x]
    if current == '.' and adj['|'] >= 3:
        current = '|'
    elif current == '|' and adj['#'] >= 3:
        current = '#'
    elif current == '#' and (adj['#'] < 1 or adj['|'] < 1):
        current = '.'
    return current

def tick(lumber):
    return [[next_state(x,y,lumber) for x in range(50)] for y in range(50)]

state = lumber
for t in range(10):
    state = tick(state)
    
string = ''.join([''.join(row) for row in state])

string.count('|') * string.count('#')

506385

## Part 2

In [169]:
visited = {}
minutes = 1000000000

def stringify(state):
    return ''.join([''.join(row) for row in state])

state = lumber
while minutes > 0:
    string = stringify(state)
    if string in visited:
        cycle_length = visited[string] - minutes
        minutes %= cycle_length
        minutes += 1
        visited = {}
    visited[string] = minutes
    state = tick(state)
    minutes -= 1

string.count('|') * string.count('#')

215404

# Day 19

## Part 1

In [197]:
with open('inputs/day19.txt') as f:
    instructions = f.read()[:-1].split('\n')
        
def addr(a,b,c,registers):
    registers[c] = registers[a] + registers[b]
    return registers
    
def addi(a,b,c,registers):
    registers[c] = registers[a] + b
    return registers
    
def mulr(a,b,c,registers):
    registers[c] = registers[a] * registers[b]
    return registers
    
def muli(a,b,c,registers):
    registers[c] = registers[a] * b
    return registers
    
def banr(a,b,c,registers):
    registers[c] = registers[a] & registers[b]
    return registers
    
def bani(a,b,c,registers):
    registers[c] = registers[a] & b
    return registers
    
def borr(a,b,c,registers):
    registers[c] = registers[a] | registers[b]
    return registers
    
def bori(a,b,c,registers):
    registers[c] = registers[a] | b
    return registers
    
def setr(a,b,c,registers):
    registers[c] = registers[a]
    return registers
    
def seti(a,b,c,registers):
    registers[c] = a
    return registers
    
def gtir(a,b,c,registers):
    registers[c] = int(a > registers[b])
    return registers
    
def gtri(a,b,c,registers):
    registers[c] = int(registers[a] > b)
    return registers
    
def gtrr(a,b,c,registers):
    registers[c] = int(registers[a] > registers[b])
    return registers
    
def eqir(a,b,c,registers):
    registers[c] = int(a == registers[b])
    return registers
    
def eqri(a,b,c,registers):
    registers[c] = int(registers[a] == b)
    return registers
    
def eqrr(a,b,c,registers):
    registers[c] = int(registers[a] == registers[b])
    return registers

ops = {'addr': addr, 
       'addi': addi,
       'mulr': mulr, 
       'muli': muli, 
       'banr': banr, 
       'bani': bani, 
       'borr': borr, 
       'bori': bori,
       'setr': setr, 
       'seti': seti, 
       'gtir': gtir,
       'gtri': gtri, 
       'gtrr': gtrr,
       'eqir': eqir, 
       'eqri': eqri,
       'eqrr': eqrr}

def process(instruction,inst_register,registers):
    op = instruction.split(' ')[0]
    args = [int(arg) for arg in instruction.split(' ')[1:]]
    
    return ops[instruction.split(' ')[0]](args[0], args[1], args[2], registers)

def execute(instructions_all, registers):
    inst_register = int(instructions_all[0].split(' ')[1])
    instructions = instructions_all[1:]
    i = 0
    while i in range(len(instructions)):
        registers[inst_register] = i
        registers = process(instructions[i],inst_register,registers)
        i = registers[inst_register] + 1
    return registers

registers = [0 for i in range(6)]
execute(instructions, registers)[0]

1248

## Part 2

In [196]:
e = 10551275
a = 0
for i in range(1,e+1):
    if e % i == 0:
        a += i
        
a

14952912

# Day 20

## Part 1

In [229]:
import queue

with open('inputs/day20.txt') as f:
    s = f.read()[:-1]
    
graph = {}
position = (0,0)
directions = {'N': (0,-1), 'S': (0,1), 'W': (-1,0), 'E': (1,0)}
    
class Node:
    def __init__(self, start, parent=None):
        self.start = start
        self.directions = []
        self.children = []
        self.parent = parent
        if parent is not None:
            parent.children.append(self)
        
    def add_direction(self,direction):
        self.directions.append(direction)        

root = Node(position)
current = root
for d in s[1:-1]:
    if d not in '(|)':
        current.add_direction(d)
        next_position = (position[0] + directions[d][0], position[1] + directions[d][1])
        if position not in graph:
            graph[position] = set()
        graph[position].add(next_position)
        position = next_position
    elif d == '(':
        current = Node(position,current)
    elif d == '|':
        position = current.start
        current = current.parent
        current = Node(position,current)
    elif d == ')':
        position = current.start
        current = current.parent
        
visited = {}
q = queue.Queue()
def bfs(state):
    position, distance = state
    if position not in visited:
        visited[position] = distance
        if position in graph:
            for door in graph[position]:
                q.put((door, distance+1))
            
q.put(((0,0), 0))
while not q.empty():
    bfs(q.get())
    
visited[max(visited, key=visited.get)]

3725

## Part 2

In [230]:
sum([1 for distance in visited.values() if distance >= 1000])

8541

# Day 21

## Part 1

In [365]:
with open('inputs/day21.txt') as f:
    instructions = f.read()[:-1].split('\n')
        
def addr(a,b,c,registers):
    registers[c] = registers[a] + registers[b]
    return registers
    
def addi(a,b,c,registers):
    registers[c] = registers[a] + b
    return registers
    
def mulr(a,b,c,registers):
    registers[c] = registers[a] * registers[b]
    return registers
    
def muli(a,b,c,registers):
    registers[c] = registers[a] * b
    return registers
    
def banr(a,b,c,registers):
    registers[c] = registers[a] & registers[b]
    return registers
    
def bani(a,b,c,registers):
    registers[c] = registers[a] & b
    return registers
    
def borr(a,b,c,registers):
    registers[c] = registers[a] | registers[b]
    return registers
    
def bori(a,b,c,registers):
    registers[c] = registers[a] | b
    return registers
    
def setr(a,b,c,registers):
    registers[c] = registers[a]
    return registers
    
def seti(a,b,c,registers):
    registers[c] = a
    return registers
    
def gtir(a,b,c,registers):
    registers[c] = int(a > registers[b])
    return registers
    
def gtri(a,b,c,registers):
    registers[c] = int(registers[a] > b)
    return registers
    
def gtrr(a,b,c,registers):
    registers[c] = int(registers[a] > registers[b])
    return registers
    
def eqir(a,b,c,registers):
    registers[c] = int(a == registers[b])
    return registers
    
def eqri(a,b,c,registers):
    registers[c] = int(registers[a] == b)
    return registers
    
def eqrr(a,b,c,registers):
    registers[c] = int(registers[a] == registers[b])
    return registers

ops = {'addr': addr, 
       'addi': addi,
       'mulr': mulr, 
       'muli': muli, 
       'banr': banr, 
       'bani': bani, 
       'borr': borr, 
       'bori': bori,
       'setr': setr, 
       'seti': seti, 
       'gtir': gtir,
       'gtri': gtri, 
       'gtrr': gtrr,
       'eqir': eqir, 
       'eqri': eqri,
       'eqrr': eqrr}

def process(instruction,inst_register,registers):
    op = instruction.split(' ')[0]
    args = [int(arg) for arg in instruction.split(' ')[1:]]
    
    return ops[instruction.split(' ')[0]](args[0], args[1], args[2], registers)

def execute(instructions_all, registers):
    inst_register = int(instructions_all[0].split(' ')[1])
    instructions = instructions_all[1:]
    i = 0
    while i in range(len(instructions)):
        registers[inst_register] = i
        registers = process(instructions[i],inst_register,registers)
        i = registers[inst_register] + 1
    return registers

registers = [0 for i in range(6)]
execute(instructions[:-2], registers)[4]

7129803

## Part 2

In [364]:
def next_halt(b):
    e = 2024736
    while b >= 1:
        c = b & 255
        e += c
        e &= 16777215
        e *= 65899
        e &= 16777215
        b //= 256
    b = e|65536
    return b,e
    

visited = set()

b = 65536
while b not in visited:
    visited.add(b)
    b,e_next = next_halt(b)
    if b not in visited:
        e = e_next

e

12284643

# Day 22

## Part 1

In [28]:
depth = 11820
target = (7,782)

geo = {(0,0): 0, target: 0}
erosion = {}
types = {}

def geologic_index(x,y):
    if (x,y) in geo:
        return geo[(x,y)]
    else:
        if y == 0:
            index = 16807*x
            geo[(x,y)] = index
            return index
        elif x == 0:
            index = 48271*y
            geo[(x,y)] = index
            return index
        else:
            index = erosion_level(x-1,y)*erosion_level(x,y-1)
            geo[(x,y)] = index
            return index

def erosion_level(x,y):
    if (x,y) in erosion:
        return erosion[(x,y)]
    else:
        level = (geologic_index(x,y) + depth) % 20183
        erosion[(x,y)] = level
        return level

def region_type(x,y):
    if (x,y) in types:
        return types[(x,y)]
    else:
        result = erosion_level(x,y) % 3
        types[(x,y)] = result
    return result

sum([region_type(x,y) for y in range(target[1]+1) for x in range(target[0]+1)])

6318

## Part 2

In [29]:
import queue

valid_gears = {0: {'C', 'T'}, 1: {'C', 'N'}, 2: {'T', 'N'}}
start = (0,0,'T')
finish = (7,782,'T')

def switch_gear(equipped,region_type,t):
    return [g for g in valid_gears[region_type] if g != equipped][0], t+7

def move_targets(x,y,equipped,t):
    possible = [(x,y-1), (x,y+1), (x-1,y), (x+1,y)]
    return [(x,y) for (x,y) in possible if x >= 0 and y >= 0 and x < 50 and y < 1000 and equipped in valid_gears[region_type(x,y)]], t+1
    
visited = {}
min_distance = None
q = queue.Queue()
def bfs(state):
    global min_distance
    
    (status,distance) = state
    if min_distance is not None and distance > min_distance:
        return
    
    if status == finish:
        if min_distance is None or min_distance > distance:
            min_distance = distance
            
    if status not in visited or visited[status] > distance:
        visited[status] = distance
        (x,y,equipped) = status
        
        new_eq, new_d = switch_gear(equipped, region_type(x,y),distance)
        new_status = (x,y,new_eq)
        if new_status not in visited or visited[new_status] > new_d:
            new_state = (new_status,new_d)
            q.put(new_state)
        
        new_positions, new_d = move_targets(x,y,equipped,distance)
        for new_pos in new_positions:
            (new_x, new_y) = new_pos
            new_status = (new_x,new_y,equipped)
            if new_status not in visited or visited[new_status] > new_d:
                new_state = (new_status,new_d)
                q.put(new_state)
            
            
q.put((start,0))
while not q.empty():
    bfs(q.get())
    
min_distance

1075

# Day 23

## Part 1

In [122]:
with open('inputs/day23.txt') as f:
    s = f.read()[:-1].split('\n')

def parse(bot):
    [position, radius] = bot[5:].split('>, r=')
    position = tuple([int(c) for c in position.split(',')])
    radius = int(radius)
    return (position, radius)

def distance(a,b):
    return abs(a[0]-b[0]) + abs(a[1]-b[1]) + abs(a[2]-b[2])

bots = [parse(bot) for bot in s]

max_radius = max(bots, key=lambda bot: bot[1])
in_range = [bot for bot in bots if distance(bot[0], max_radius[0]) <= max_radius[1]]

len(in_range)

481

## Part 2

In [125]:
def box_dist(box, bot):
    (bot_x, bot_y, bot_z) = bot
    dist = 0
    dist += box.x - bot_x if bot_x <= box.x else bot_x - (box.x + box.size - 1)
    dist += box.y - bot_y if bot_y <= box.y else bot_y - (box.y + box.size - 1)
    dist += box.z - bot_z if bot_z <= box.z else bot_z - (box.z + box.size - 1)
    
    return dist

def box_in_range(box):
    return len([bot for bot in bots if box_dist(box, bot[0]) <= bot[1]])

class Box:
    def __init__(self,x,y,z,size):
        self.x = x
        self.y = y
        self.z = z
        self.size = size
        self.distance = abs(x) + abs(y) + abs(z)
        self.in_range = box_in_range(self)
        
def split(box):
    split_x = box.x + box.size//2
    split_y = box.y + box.size//2
    split_z = box.z + box.size//2
    
    boxes.add(Box(box.x, box.y, box.z, box.size//2))
    boxes.add(Box(box.x, box.y, split_z, box.size//2))
    boxes.add(Box(box.x, split_y, box.z, box.size//2))
    boxes.add(Box(box.x, split_y, split_z, box.size//2))
    boxes.add(Box(split_x, box.y, box.z, box.size//2))
    boxes.add(Box(split_x, box.y, split_z, box.size//2))
    boxes.add(Box(split_x, split_y, box.z, box.size//2))
    boxes.add(Box(split_x, split_y, split_z, box.size//2))

min_x = min([bot[0][0] for bot in bots] + [0])
max_x = max([bot[0][0] for bot in bots] + [0])
min_y = min([bot[0][1] for bot in bots] + [0])
max_y = max([bot[0][1] for bot in bots] + [0])
min_z = min([bot[0][2] for bot in bots] + [0])
max_z = max([bot[0][2] for bot in bots] + [0])

box_size = 1
while min_x + box_size < max_x or min_y + box_size < max_y or min_z + box_size < max_z:
    box_size *= 2

box = Box(min_x, min_y, min_z, box_size)
boxes = {box}

def get_box():
    max_bots = max(boxes, key=lambda box: box.in_range).in_range
    most_bots = [box for box in boxes if box.in_range == max_bots]
    most_bots.sort(key=lambda box: box.size)
    most_bots.sort(key=lambda box: box.distance)
    
    box = most_bots[0]
    boxes.remove(box)
    return box
        
while box.size != 1:
    box = get_box()
    split(box)
    
box.distance

47141479

# Day 24

## Part 1

In [345]:
with open('inputs/day24.txt') as f:
    s = f.read()[:-1].split('\n')
    
class Group:
    def __init__(self, units, hp, dmg, dmg_type, init, team, weak, immune):
        self.units = units
        self.hp = hp
        self.dmg = dmg
        self.dmg_type = dmg_type
        self.init = init
        self.team = team
        self.weak = weak
        self.immune = immune
        self.enemy = 'Infection' if team == 'Immune' else 'Immune'
        self.target = None
        self.attacking = None
        
    def __repr__(self):
        return ', '.join([self.team, str(self.init), str(self.units)])
        
    def power(self):
        return max(self.units ,0) * self.dmg
        
    def attack_effect(self, dmg, dmg_type):
        if dmg_type in self.immune:
            return 0
        if dmg_type in self.weak:
            return dmg*2
        return dmg
    
    def damage(self, dmg, dmg_type):
        actual_dmg = self.attack_effect(dmg, dmg_type)
        self.units -= actual_dmg // self.hp
        if self.units <= 0:
            self.die()
        return actual_dmg // self.hp > 0
            
    def die(self):
        groups.remove(self)
        
    def pick_target(self):
        targets = [group for group in groups if group.team == self.enemy and group.attacking is None]
        if self in groups and len(targets) > 0:
            targets.sort(reverse=True, key=lambda group: group.init)
            targets.sort(reverse=True, key=lambda group: group.power())
            targets.sort(reverse=True, key=lambda group: group.attack_effect(self.power(), self.dmg_type))
            target = targets[0]
            if target.attack_effect(self.power(), self.dmg_type) > 0:
                self.target = target
                self.target.attacking = self
    
    def attack(self):
        if self in groups and self.target in groups:
            return self.target.damage(self.power(), self.dmg_type)
        return False
        
        
def tick():
    for group in groups:
        group.target = None
        group.attacking = None
    order = [group for group in groups]
    order.sort(reverse=True, key=lambda group: group.init)
    order.sort(reverse=True, key=lambda group: group.power())
    for group in order:
        group.pick_target()
    order = [group for group in order if group.target is not None]
    order.sort(reverse=True, key=lambda group: group.init)
    attacked = False
    for group in order:
        if group in groups:
            attacked |= group.attack()
    return attacked

def parse_modifier(mod):
    weak = []
    immune = []
    for modifier in mod.split('; '):
        if 'weak' in modifier:
            weak = modifier[8:].split(', ')
        else:
            immune = modifier[10:].split(', ')
    return weak, immune
            
def parse(group, team, boost=0):
    units = int(group.split(' ')[0])
    hp = int(group.split(' ')[4])
    weak = []
    immune = []
    if '(' in group:
        weak, immune = parse_modifier(group[group.index('(')+1:group.index(')')])
    group = group[group.index('attack'):]
    dmg = int(group.split(' ')[3])
    if team == 'Immune':
        dmg += boost
    dmg_type = group.split(' ')[4]
    init = int(group.split(' ')[-1])
    return Group(units, hp, dmg, dmg_type, init, team, weak, immune)
    
            
groups = []
for line in s:
    if 'Immune' in line:
        team = 'Immune'
    elif 'Infection' in line:
        team = 'Infection'
    elif 'units' in line:
        groups.append(parse(line, team))

progress = True
while len({group.team for group in groups}) > 1 and progress:
    progress = tick()

sum([group.units for group in groups])

20340

## Part 2

In [354]:
boost = 0
progress = True
winner = 'Infection'

while not (winner == 'Immune' and progress):
    boost += 1
    groups = []
    for line in s:
        if 'Immune' in line:
            team = 'Immune'
        elif 'Infection' in line:
            team = 'Infection'
        elif 'units' in line:
            groups.append(parse(line, team, boost))

    progress = True
    while len({group.team for group in groups}) > 1 and progress:
        progress = tick()
    winner = groups[0].team
    
sum([group.units for group in groups])

3862

# Day 25

In [379]:
with open('inputs/day25.txt') as f:
    s = f.read()[:-1].split('\n')
    
points = [(int(p.split(',')[0]), int(p.split(',')[1]), int(p.split(',')[2]), int(p.split(',')[3])) for p in s]

def manhattan(p0, p1):
    return abs(p0[0]-p1[0]) + abs(p0[1]-p1[1]) + abs(p0[2]-p1[2]) + abs(p0[3]-p1[3])

visited = set()
constellations = {}
while len(points) > 0:
    point = points[0]
    constellation = len(constellations)
    constellations[constellation] = {point}
    visited.add(point)
    new_points = [point]
    while len(new_points) != 0:
        new_points = []
        for p0 in constellations[constellation]:
            for p1 in points:
                if p1 not in visited and manhattan(p0,p1) <= 3:
                    new_points.append(p1)
        constellations[constellation].update(new_points)
        visited.update(new_points)
    points = [point for point in points if point not in visited]
        
len(constellations)

390